In [2]:

#version 3 of test_single_trial_RAM_DISK.py with updated VOLPY from old single trial .py file
# TO RUN: conda activate caiman
# # python C:\Users\ICNLab\caiman_data\test_single_trial_RAM_DISK_3.py C:\Users\ICNLab\caiman_data\testdata\testdata\FOV1_T2RAM2\FOV1_T2.tsm

fname = r'C:\Users\ICNLab\caiman_data\testdata\testdata\FOV1_T2RAM2\FOV1_T2.tsm'
#fname = r'C:\Users\ICNLab\caiman_data\testdata\testdata\FOV3_T3new\FOV3_T3.tsm'

print("Importing packages and Initializing...")
from base64 import b64encode
import cv2
import glob
import h5py
import imageio
from IPython import get_ipython
from IPython.display import HTML, display, clear_output
import logging
import matplotlib.pyplot as plt
#matplotlib.use("Agg")  # non-interactive backend
import numpy as np
import os
import tensorflow as tf
from pathlib import Path
from PIL import Image

#import to cover extras from single_trial.py
import gc
import scipy.io
from scipy import stats
from scipy.signal import butter, lfilter
from scipy.signal import savgol_filter
import sys
import mat73
import pandas as pd


from pathlib import Path

try:
    cv2.setNumThreads(0)
except:
    pass

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
        get_ipython().run_line_magic('matplotlib', 'qt')
except NameError:
    pass

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.utils.utils import download_demo, download_model
from caiman.source_extraction.volpy import utils
from caiman.source_extraction.volpy.volparams import volparams
from caiman.source_extraction.volpy.volpy import VOLPY
from caiman.source_extraction.volpy.mrcnn import visualize, neurons
import caiman.source_extraction.volpy.mrcnn.model as modellib
from caiman.summary_images import local_correlations_movie_offline
from caiman.summary_images import mean_image
from caiman.paths import caiman_datadir
from caiman.summary_images import local_correlations_movie_in_memory
import gc


logging.basicConfig(format=
                    "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s]" \
                    "[%(process)d] %(message)s",
                    level=logging.ERROR)

##
#fname = r'C:\Users\ICNLab\caiman_data\testdata\testdata\FOV1_T2RAM2\FOV1_T2.tsm'
fr = 640
print(fname, fr)


##
# Cleanup R:/ drive
print("Cleaning up R:/ drive...")
def safe_close_mmap(arr):
    try:
        if hasattr(arr, "base") and hasattr(arr.base, "close"):
            arr.base.close()
    except Exception as e:
        print("close failed:", e)



Importing packages and Initializing...
C:\Users\ICNLab\caiman_data\testdata\testdata\FOV1_T2RAM2\FOV1_T2.tsm 640
Cleaning up R:/ drive...


In [ ]:

# 1. Delete any Python references to memmaps pointing to R:/
try:
    safe_close_mmap(Yr)  # or whatever your memmap object is called
except NameError:
    pass

try:
    safe_close_mmap(mmap_file_rig)  # or whatever your memmap object is called
except NameError:
    pass

gc.collect()  # force Python to release the memory mapping

# 2. Delete all files in R:/
for f in Path(r'R:/').glob('*'):
    if f.is_file():
        f.unlink()
print("Cleared all files from R:/")


##
pw_rigid = False  # flag for pw-rigid motion correction
gsig_filt = (3, 3)  # size of filter, in general gSig (see below),
# change this one if algorithm does not work
max_shifts = (5, 5)  # maximum allowed rigid shift
strides = (48, 48)  # start a new patch for pw-rigid motion correction every x pixels
overlaps = (24, 24)  # overlap between paths (size of patch strides+overlaps)
max_deviation_rigid = 3  # maximum deviation allowed for patch with respect to rigid shifts
border_nan = 'copy'
use_cuda = True

opts_dict = {
    'fnames': fname,
    'fr': fr,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'gSig_filt': gsig_filt,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': border_nan,
    'use_cuda': use_cuda
}

opts = volparams(params_dict=opts_dict)

##
print("Loading data...")
m_orig = cm.load(fname)
ds_ratio = 0.2

##
c, dview, n_processes = cm.cluster.setup_cluster(
            backend='local', n_processes=None, single_thread=False)

##
print("Motion correction...")
mc = MotionCorrect(fname, dview=dview, **opts.get_group('motion'))
mc.motion_correct(save_movie=True)
#about 2.3 minutes for 12800 frames (2m 13-21 s)
print("Done.")

##
print("Loading corrected movie...")
m_rig = cm.load(mc.mmap_file) # 11s
ds_ratio = 0.2
print("Done.")


##
print("Saving stabilized movie to RAM-disk...")
# Path to RAM disk memmap
p = Path(fname)
ram_path = Path(r'R:/') / f"{p.stem}_rig__d1_{m_rig.shape[1]}_d2_{m_rig.shape[2]}_d3_1_order_C_frames_{m_rig.shape[0]}.mmap"
ram_path = str(ram_path).replace("/", "\\")

# Create memmap in RAM-disk with same shape as m_rig
mmap_file_rig = np.memmap(ram_path, dtype='float32', mode='w+', shape=m_rig.shape, order='F') #Was C before

# Copy stabilized movie data into memmap
mmap_file_rig[:] = m_rig[:]

# Flush to make sure data is written
mmap_file_rig.flush()

mmap_list = [mmap_file_rig]

print("Saved stabilized memmap to RAM-disk:", ram_path)

##

print("Computing mean and correlation images...")
img = np.mean(m_rig, axis=0)
img = (img-np.mean(img))/np.std(img)

gaussian_blur = False


In [ ]:
print(mc.mmap_file[0])



R:/FOV1_T2_rig__d1_512_d2_512_d3_1_order_F_frames_12800.mmap


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt




In [ ]:

# ===============================
# 1. Parameters
# ===============================
VIDEO_PATH = "video.mmap"   # path to your mmap file
SHAPE = (12800, 512, 512)
DTYPE = np.float32

FRAME_RATE = 640.0  # Hz (EDIT THIS IF NEEDED)
BANDPASS = (5, 300) # Hz #70,300

WINDOW = 3
HALF = WINDOW // 2


In [11]:

# ===============================
# 2. Load memory-mapped video
# ===============================
video = np.memmap(
    mc.mmap_file[0],
    dtype=DTYPE,
    mode="r",
    shape=(SHAPE[0], SHAPE[2], SHAPE[1]),
    order="C"
).swapaxes(1, 2)

T, H, W = video.shape
print(f"Loaded video: {video.shape}")


Loaded video: (12800, 512, 512)


In [12]:

# # ===============================
# # 3. Band-pass filter design
# # ===============================
# def bandpass_filter(data, fs, low, high, order=3):
#     nyq = 0.5 * fs
#     b, a = butter(order, [low / nyq, high / nyq], btype="band")
#     return filtfilt(b, a, data, axis=0)

# ===============================
# 3. High-pass filter (bandpass-compatible API)
# ===============================
def bandpass_filter(data, fs, low, high=None, order=3):
    """
    High-pass filter using the 'low' cutoff.
    The 'high' argument is accepted for API compatibility but ignored.
    """
    nyq = 0.5 * fs
    b, a = butter(order, low / nyq, btype="high")
    return filtfilt(b, a, data, axis=0)


In [ ]:

# ===============================
# 4. Example: filter one pixel (DEBUG)
# ===============================
# y, x = 256, 256
# raw_trace = video[:, y, x]
# filt_trace = bandpass_filter(raw_trace, FRAME_RATE, *BANDPASS)

# plt.figure(figsize=(8, 3))
# plt.plot(raw_trace[:2000], label="Raw", alpha=0.5)
# plt.plot(filt_trace[:2000], label="Band-passed", lw=2)
# plt.title("Single pixel trace (first 2 seconds)")
# plt.xlabel("Frame")
# plt.legend()
# plt.tight_layout()
# plt.show()



# ===============================
# 4. Interactive pixel selection (DEBUG)
# ===============================
fig, ax = plt.subplots(figsize=(6, 6))

first_frame = video[0]
im = ax.imshow(first_frame, cmap="gray")
ax.set_title("Click on a pixel to inspect its time trace")
ax.axis("off")
plt.colorbar(im, ax=ax, fraction=0.046)

coords = []

def onclick(event):
    if event.inaxes != ax:
        return

    x = int(event.xdata)
    y = int(event.ydata)
    coords.append((y, x))

    ax.plot(x, y, 'ro')
    fig.canvas.draw()

    print(f"Selected pixel: (y={y}, x={x})")

    # Extract and filter trace
    raw_trace = video[:, y, x]
    filt_trace = bandpass_filter(
        raw_trace, FRAME_RATE, *BANDPASS
    )

    # Plot traces
    plt.figure(figsize=(8, 3))
    plt.plot(raw_trace[:2000], label="Raw", alpha=0.5)
    plt.plot(filt_trace[:2000], label="Band-passed", lw=2)
    plt.xlabel("Frame")
    plt.ylabel("Fluorescence")
    plt.title(f"Pixel (y={y}, x={x}) – first 2 seconds")
    plt.legend()
    plt.tight_layout()
    plt.show()

cid = fig.canvas.mpl_connect('button_press_event', onclick)
plt.show()



Selected pixel: (y=320, x=210)
Selected pixel: (y=179, x=212)


In [ ]:
# ===============================
# 4. Interactive circular ROI selection (POC)
# ===============================

# DISPLAY_SECONDS = 20  # seconds of data to plot

# fig, ax = plt.subplots(figsize=(6, 6))

# first_frame = video[0]
# im = ax.imshow(first_frame, cmap="gray")
# ax.set_title("Click: set center | Scroll: change radius | Enter: extract")
# ax.axis("off")
# plt.colorbar(im, ax=ax, fraction=0.046)

# roi_center = None
# roi_radius = 5  # initial radius (pixels)
# circle_artist = None




# ===============================
# Parameters
# ===============================
DISPLAY_SECONDS = 20
TILE_SIZE = 8
roi_radius = 5  # initial radius (pixels)
roi_center = None
circle_artist = None

# ===============================
# First frame & coherence image
# ===============================
first_frame = video[0]  # shape (H, W)

fig, ax = plt.subplots(figsize=(6, 6))
im = ax.imshow(first_frame, cmap="gray")
ax.set_title("Click: set center | Scroll: change radius | Enter: extract")
ax.axis("off")
plt.colorbar(im, ax=ax, fraction=0.046)

# Overlay coherence image
vmax = np.percentile(spike_norm, 99)
overlay = ax.imshow(spike_norm, cmap="viridis", alpha=0.5, vmin=0, vmax=vmax)





def draw_circle():
    global circle_artist
    if circle_artist is not None:
        circle_artist.remove()

    circle_artist = plt.Circle(
        (roi_center[1], roi_center[0]),
        roi_radius,
        edgecolor="red",
        facecolor="none",
        lw=2
    )
    ax.add_patch(circle_artist)
    fig.canvas.draw_idle()


def onclick(event):
    global roi_center
    if event.inaxes != ax:
        return

    roi_center = (int(event.ydata), int(event.xdata))
    draw_circle()
    print(f"ROI center: (y={roi_center[0]}, x={roi_center[1]})")


def onscroll(event):
    global roi_radius
    if roi_center is None:
        return

    if event.button == 'up':
        roi_radius += 1
    elif event.button == 'down' and roi_radius > 1:
        roi_radius -= 1

    draw_circle()
    print(f"ROI radius: {roi_radius} px")


def onkeypress(event):
    if event.key != 'enter' or roi_center is None:
        return

    y0, x0 = roi_center

    yy, xx = np.ogrid[:H, :W]
    mask = (yy - y0) ** 2 + (xx - x0) ** 2 <= roi_radius ** 2
    n_pixels = mask.sum()

    print(f"Extracting {n_pixels} pixels from ROI")

    # Extract summed signal
    roi_trace = video[:, mask].sum(axis=1)

    # Band-pass filter
    filt_trace = bandpass_filter(
        roi_trace, FRAME_RATE, *BANDPASS
    )

    # Plot
    n_show = min(int(DISPLAY_SECONDS * FRAME_RATE), len(roi_trace))
    t = np.arange(n_show) / FRAME_RATE

    plt.figure(figsize=(10, 4))
    plt.plot(t, roi_trace[:n_show], label="Summed ROI (raw)", alpha=0.5)
    plt.plot(t, filt_trace[:n_show], label="Band-passed", lw=2)
    plt.xlabel("Time (s)")
    plt.ylabel("Summed fluorescence")
    plt.title(
        f"Circular ROI (r={roi_radius}px, n={n_pixels})"
    )
    plt.legend()
    plt.tight_layout()
    plt.show()


# Connect callbacks
fig.canvas.mpl_connect("button_press_event", onclick)
fig.canvas.mpl_connect("scroll_event", onscroll)
fig.canvas.mpl_connect("key_press_event", onkeypress)

plt.show()


ROI center: (y=149, x=273)
ROI radius: 6 px
ROI radius: 7 px
ROI radius: 8 px
ROI radius: 9 px
ROI radius: 10 px
ROI radius: 11 px
ROI radius: 12 px
ROI radius: 13 px
ROI radius: 14 px
Extracting 613 pixels from ROI


In [13]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# ===============================
# Parameters
# ===============================
TILE_SIZE = 4
H, W = 512, 512
FRAME_RATE = FRAME_RATE   # already defined
BANDPASS = BANDPASS       # (low, high), high ignored
DISPLAY_CLIP = 99         # percentile for visualization

# ===============================
# Robust spike metric
# ===============================
def spikiness_metric(trace, z_thresh=3.0):
    """
    Estimate amount of outlying activity in a trace.
    Uses robust z-score (MAD-based) and measures tail mass.
    """
    trace = trace - np.median(trace)
    mad = np.median(np.abs(trace)) + 1e-9
    z = trace / mad

    # Fraction of samples beyond threshold
    spike_fraction = np.mean(np.abs(z) > z_thresh)

    # Mean excess magnitude beyond threshold
    spike_energy = np.mean(np.abs(z[np.abs(z) > z_thresh])) if np.any(np.abs(z) > z_thresh) else 0.0

    return spike_fraction + 0.1 * spike_energy


# ===============================
# Output map (tile-resolution)
# ===============================
n_tiles_y = H // TILE_SIZE
n_tiles_x = W // TILE_SIZE

tile_spike_map = np.zeros((n_tiles_y, n_tiles_x), dtype=np.float32)

# ===============================
# Main loop
# ===============================
with tqdm(total=n_tiles_y * n_tiles_x, desc="Analyzing tiles") as pbar:
    for ty in range(n_tiles_y):
        for tx in range(n_tiles_x):

            y0 = ty * TILE_SIZE
            y1 = y0 + TILE_SIZE
            x0 = tx * TILE_SIZE
            x1 = x0 + TILE_SIZE

            # Extract tile: shape (T, 16, 16)
            tile = video[:, y0:y1, x0:x1]

            # Sum all pixels
            tile_trace = tile.reshape(T, -1).sum(axis=1)

            # High-pass filter (bandpass-compatible)
            tile_trace_filt = bandpass_filter(
                tile_trace, FRAME_RATE, *BANDPASS
            )

            # Compute spikiness
            tile_spike_map[ty, tx] = spikiness_metric(tile_trace_filt)

            pbar.update(1)

# ===============================
# Expand tile map back to image resolution
# ===============================
spike_image = np.repeat(
    np.repeat(tile_spike_map, TILE_SIZE, axis=0),
    TILE_SIZE, axis=1
)

# # ===============================
# # Visualization
# # ===============================
# vmax = np.percentile(spike_image, DISPLAY_CLIP)

# plt.figure(figsize=(6, 6))
# plt.imshow(spike_image, cmap="hot", vmin=0, vmax=vmax)
# plt.title("Grid-based spike activity ("+str(TILE_SIZE)+"×"+str(TILE_SIZE)+" tiles)")
# plt.colorbar(label="Spikiness score")
# plt.axis("off")
# plt.tight_layout()
# plt.show()

# ===============================
# Visualization (normalized)
# ===============================

# Normalize spike image to [0, 1]
spike_norm = spike_image.astype(np.float32)
spike_norm -= spike_norm.min()
spike_norm /= (spike_norm.max() + 1e-9)  # avoid division by zero

# Optionally clip at DISPLAY_CLIP percentile for contrast
vmax = np.percentile(spike_norm, DISPLAY_CLIP / 100 * 1.0)

plt.figure(figsize=(6, 6))
plt.imshow(spike_norm, cmap="hot", vmin=0, vmax=vmax)
plt.title(f"Grid-based spike activity ({TILE_SIZE}×{TILE_SIZE} tiles)")
plt.colorbar(label="Spikiness score")
plt.axis("off")
plt.tight_layout()
plt.show()


Analyzing tiles: 100%|██████████| 16384/16384 [00:38<00:00, 429.86it/s]


In [14]:
# ===============================
# Visualization
# ===============================
vmax = np.percentile(spike_image, DISPLAY_CLIP)

plt.figure(figsize=(6, 6))
plt.imshow(spike_image, cmap="hot", vmin=0, vmax=vmax)
plt.title("Grid-based spike activity ("+str(TILE_SIZE)+"×"+str(TILE_SIZE)+" tiles)")
plt.colorbar(label="Spikiness score")
plt.axis("off")
plt.tight_layout()
plt.show()

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# ===============================
# Parameters
# ===============================
TILE_SIZE = 4
H, W = 512, 512
FRAME_RATE = FRAME_RATE
BANDPASS = BANDPASS      # (low, high), high ignored
DISPLAY_CLIP = 99

# ===============================
# Coherence metric
# ===============================
def coherence_metric(tile_filt):
    """
    tile_filt: shape (T, Npix)
    Returns mean pixel-to-tile correlation.
    """
    # Tile reference (subthreshold signals sum coherently)
    ref = tile_filt.mean(axis=1)

    ref -= ref.mean()
    ref_std = ref.std() + 1e-9

    # Normalize reference
    ref /= ref_std

    # Normalize pixels
    pix = tile_filt - tile_filt.mean(axis=0)
    pix /= (pix.std(axis=0) + 1e-9)

    # Correlation with reference
    corr = np.mean(ref[:, None] * pix, axis=0)

    # Use mean absolute correlation as coherence
    return np.mean(np.abs(corr))


# ===============================
# Output tile map
# ===============================
n_tiles_y = H // TILE_SIZE
n_tiles_x = W // TILE_SIZE

tile_coherence_map = np.zeros((n_tiles_y, n_tiles_x), dtype=np.float32)

# ===============================
# Main loop
# ===============================
with tqdm(total=n_tiles_y * n_tiles_x, desc="Computing coherence") as pbar:
    for ty in range(n_tiles_y):
        for tx in range(n_tiles_x):

            y0 = ty * TILE_SIZE
            y1 = y0 + TILE_SIZE
            x0 = tx * TILE_SIZE
            x1 = x0 + TILE_SIZE

            # Extract tile: (T, 16, 16)
            tile = video[:, y0:y1, x0:x1]
            tile = tile.reshape(T, -1)

            # High-pass filter all pixels independently
            tile_filt = bandpass_filter(
                tile, FRAME_RATE, *BANDPASS
            )

            # Compute coherence
            tile_coherence_map[ty, tx] = coherence_metric(tile_filt)

            pbar.update(1)

# ===============================
# Expand to image resolution
# ===============================
coherence_image = np.repeat(
    np.repeat(tile_coherence_map, TILE_SIZE, axis=0),
    TILE_SIZE, axis=1
)

# ===============================
# Visualization
# ===============================
vmax = np.percentile(coherence_image, DISPLAY_CLIP)

plt.figure(figsize=(6, 6))
plt.imshow(coherence_image, cmap="viridis", vmin=0, vmax=vmax)
plt.title("Grid-based subthreshold coherence ("+str(TILE_SIZE)+"×"+str(TILE_SIZE)+" tiles)")
plt.colorbar(label="Mean |pixel–tile correlation|")
plt.axis("off")
plt.tight_layout()
plt.show()

# # ===============================
# # Visualization (normalized)
# # ===============================

# # Normalize coherence image to [0, 1] for visualization
# coh_norm = coherence_image.astype(np.float32)
# coh_norm -= coh_norm.min()
# coh_norm /= (coh_norm.max() + 1e-9)  # avoid division by zero

# # Optionally clip at DISPLAY_CLIP percentile for display contrast
# vmax = np.percentile(coh_norm, DISPLAY_CLIP / 100 * 1.0)

# plt.figure(figsize=(6, 6))
# plt.imshow(coh_norm, cmap="viridis", vmin=0, vmax=vmax)
# plt.title(f"Grid-based subthreshold coherence ({TILE_SIZE}×{TILE_SIZE} tiles)")
# plt.colorbar(label="Mean |pixel–tile correlation|")
# plt.axis("off")
# plt.tight_layout()
# plt.show()


Computing coherence: 100%|██████████| 16384/16384 [01:53<00:00, 144.78it/s]


In [16]:
# ===============================
# Visualization (normalized)
# ===============================

# Normalize coherence image to [0, 1] for visualization
coh_norm = coherence_image.astype(np.float32)
coh_norm -= coh_norm.min()
coh_norm /= (coh_norm.max() + 1e-9)  # avoid division by zero

# Optionally clip at DISPLAY_CLIP percentile for display contrast
vmax = np.percentile(coh_norm, DISPLAY_CLIP / 100 * 1.0)

plt.figure(figsize=(6, 6))
plt.imshow(coh_norm, cmap="viridis", vmin=0, vmax=vmax)
plt.title(f"Grid-based subthreshold coherence ({TILE_SIZE}×{TILE_SIZE} tiles)")
plt.colorbar(label="Mean |pixel–tile correlation|")
plt.axis("off")
plt.tight_layout()
plt.show()


In [12]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# ===============================
# Parameters
# ===============================
TILE_SIZE = 8
H, W = 512, 512
FRAME_RATE = FRAME_RATE
BANDPASS = BANDPASS
DISPLAY_CLIP = 99

# ===============================
# Coherence metric (vectorized)
# ===============================
def coherence_metric_batch(tile_filt):
    """
    tile_filt: (T, Npix)
    """
    ref = tile_filt.mean(axis=1)
    ref -= ref.mean()
    ref /= (ref.std() + 1e-9)

    pix = tile_filt - tile_filt.mean(axis=0)
    pix /= (pix.std(axis=0) + 1e-9)

    corr = np.mean(ref[:, None] * pix, axis=0)
    return np.mean(np.abs(corr))


# ===============================
# Output tile map
# ===============================
n_tiles_y = H // TILE_SIZE
n_tiles_x = W // TILE_SIZE

tile_coherence_map = np.zeros((n_tiles_y, n_tiles_x), dtype=np.float32)

# ===============================
# Main loop (fast)
# ===============================
for ty in tqdm(range(n_tiles_y), desc="Computing coherence"):
    y0 = ty * TILE_SIZE
    y1 = y0 + TILE_SIZE

    # Pull full row of tiles at once → contiguous memory
    row = video[:, y0:y1, :].reshape(video.shape[0], TILE_SIZE, n_tiles_x, TILE_SIZE)
    row = row.transpose(0, 1, 3, 2).reshape(video.shape[0], -1, n_tiles_x)

    for tx in range(n_tiles_x):
        tile = row[:, :, tx]

        tile_filt = bandpass_filter(tile, FRAME_RATE, *BANDPASS)
        #tile_filt = tile.copy()
        tile_coherence_map[ty, tx] = coherence_metric_batch(tile_filt)

# ===============================
# Expand to image resolution
# ===============================
coherence_image = np.repeat(
    np.repeat(tile_coherence_map, TILE_SIZE, axis=0),
    TILE_SIZE, axis=1
)

# ===============================
# Visualization
# ===============================
vmax = np.percentile(coherence_image, DISPLAY_CLIP)

plt.figure(figsize=(6, 6))
plt.imshow(coherence_image, cmap="viridis", vmin=0, vmax=vmax)
plt.title("Grid-based subthreshold coherence (8×8 tiles)")
plt.colorbar(label="Mean |pixel–tile correlation|")
plt.axis("off")
plt.tight_layout()
plt.show()


Computing coherence: 100%|██████████| 64/64 [01:51<00:00,  1.75s/it]


In [11]:
import numpy as np
import cupy as cp        # GPU version of numpy
import matplotlib.pyplot as plt
from tqdm import tqdm

# ===============================
# Parameters
# ===============================
TILE_SIZE = 8
H, W = 512, 512
FRAME_RATE = FRAME_RATE
BANDPASS = BANDPASS
DISPLAY_CLIP = 99

# ===============================
# Coherence metric (GPU)
# ===============================
def coherence_metric_batch_gpu(tile_filt):
    """
    tile_filt: cp.array of shape (T, Npix)
    """
    ref = cp.mean(tile_filt, axis=1)
    ref -= cp.mean(ref)
    ref /= (cp.std(ref) + 1e-9)

    pix = tile_filt - cp.mean(tile_filt, axis=0)
    pix /= (cp.std(pix, axis=0) + 1e-9)

    corr = cp.mean(ref[:, None] * pix, axis=0)
    return float(cp.mean(cp.abs(corr)))  # convert back to host float

# ===============================
# Output tile map
# ===============================
n_tiles_y = H // TILE_SIZE
n_tiles_x = W // TILE_SIZE
tile_coherence_map = np.zeros((n_tiles_y, n_tiles_x), dtype=np.float32)

# ===============================
# Main loop (row-wise, GPU compute)
# ===============================
for ty in tqdm(range(n_tiles_y), desc="Computing coherence"):
    y0 = ty * TILE_SIZE
    y1 = y0 + TILE_SIZE

    # Pull full row of tiles → contiguous memory
    row = video[:, y0:y1, :].reshape(video.shape[0], TILE_SIZE, n_tiles_x, TILE_SIZE)
    row = row.transpose(0, 1, 3, 2).reshape(video.shape[0], -1, n_tiles_x)

    for tx in range(n_tiles_x):
        tile = row[:, :, tx]

        # Copy tile to GPU
        tile_gpu = cp.asarray(tile)

        # Compute coherence on GPU
        tile_coherence_map[ty, tx] = coherence_metric_batch_gpu(tile_gpu)

        # Free GPU memory immediately
        del tile_gpu
        cp._default_memory_pool.free_all_blocks()

# ===============================
# Expand to image resolution
# ===============================
coherence_image = np.repeat(
    np.repeat(tile_coherence_map, TILE_SIZE, axis=0),
    TILE_SIZE, axis=1
)

# ===============================
# Visualization
# ===============================
vmax = np.percentile(coherence_image, DISPLAY_CLIP)

plt.figure(figsize=(6, 6))
plt.imshow(coherence_image, cmap="viridis", vmin=0, vmax=vmax)
plt.title("Grid-based subthreshold coherence (8×8 tiles, GPU)")
plt.colorbar(label="Mean |pixel–tile correlation|")
plt.axis("off")
plt.tight_layout()
plt.show()


Computing coherence: 100%|██████████| 64/64 [00:46<00:00,  1.36it/s]


In [ ]:

# ===============================
# 5. Correlation computation
# ===============================
correlation_map = np.zeros((H, W), dtype=np.float32)
count_map = np.zeros((H, W), dtype=np.int32)

for y in range(HALF, H - HALF):
    for x in range(HALF, W - HALF):

        # Extract 3x3 pixel signals
        patch = video[:, y-HALF:y+HALF+1, x-HALF:x+HALF+1]
        patch = patch.reshape(T, -1)

        # Band-pass filter each pixel independently
        patch_filt = bandpass_filter(
            patch, FRAME_RATE, *BANDPASS
        )

        # Correlation matrix (9x9)
        C = np.corrcoef(patch_filt.T)

        # Mean off-diagonal correlation
        mean_corr = (
            np.sum(C) - np.trace(C)
        ) / (C.size - C.shape[0])

        # Assign correlation to all 9 pixels
        correlation_map[y-HALF:y+HALF+1,
                        x-HALF:x+HALF+1] += mean_corr
        count_map[y-HALF:y+HALF+1,
                  x-HALF:x+HALF+1] += 1

# Normalize overlapping contributions
correlation_map /= np.maximum(count_map, 1)


Computing coherence map with 28 parallel jobs...


  5%|▌         | 56/1024 [00:03<01:01, 15.62it/s] [Parallel(n_jobs=28)]: Done   5 tasks      | elapsed:    3.2s
[Parallel(n_jobs=28)]: Done  16 tasks      | elapsed:    3.8s
[Parallel(n_jobs=28)]: Done  29 tasks      | elapsed:    4.5s
  8%|▊         | 84/1024 [00:04<00:55, 17.03it/s][Parallel(n_jobs=28)]: Done  42 tasks      | elapsed:    5.3s
[Parallel(n_jobs=28)]: Done  57 tasks      | elapsed:    5.9s
 14%|█▎        | 140/1024 [00:07<00:45, 19.52it/s][Parallel(n_jobs=28)]: Done  89 tasks      | elapsed:    7.2s
[Parallel(n_jobs=28)]: Done 106 tasks      | elapsed:    7.9s
 19%|█▉        | 196/1024 [00:09<00:37, 22.14it/s][Parallel(n_jobs=28)]: Done 144 tasks      | elapsed:    9.4s
[Parallel(n_jobs=28)]: Done 165 tasks      | elapsed:   10.2s
 33%|███▎      | 336/1024 [00:14<00:27, 25.38it/s][Parallel(n_jobs=28)]: Done 282 tasks      | elapsed:   14.7s
[Parallel(n_jobs=28)]: Done 309 tasks      | elapsed:   16.1s
100%|██████████| 1024/1024 [00:44<00:00, 22.94it/s]
[Parallel(n_jobs=

In [ ]:
from tqdm import tqdm

# ===============================
# 5. Correlation computation (with progress bar)
# ===============================
correlation_map = np.zeros((H, W), dtype=np.float32)
count_map = np.zeros((H, W), dtype=np.int32)

total_pixels = (H - 2*HALF) * (W - 2*HALF)

with tqdm(total=total_pixels, desc="Computing 3×3 correlations") as pbar:
    for y in range(HALF, H - HALF):
        for x in range(HALF, W - HALF):

            # Extract 3x3 pixel signals
            patch = video[:, y-HALF:y+HALF+1, x-HALF:x+HALF+1]
            patch = patch.reshape(T, -1)

            # Band-pass filter each pixel independently
            patch_filt = bandpass_filter(
                patch, FRAME_RATE, *BANDPASS
            )

            # Correlation matrix (9x9)
            C = np.corrcoef(patch_filt.T)

            # Mean off-diagonal correlation
            mean_corr = (
                np.sum(C) - np.trace(C)
            ) / (C.size - C.shape[0])

            # Accumulate correlation to pixels
            correlation_map[y-HALF:y+HALF+1,
                            x-HALF:x+HALF+1] += mean_corr
            count_map[y-HALF:y+HALF+1,
                      x-HALF:x+HALF+1] += 1

            pbar.update(1)

# Normalize overlapping contributions
correlation_map /= np.maximum(count_map, 1)


In [ ]:

# ===============================
# 6. Debug: correlation matrix of one patch
# ===============================
plt.figure(figsize=(4, 4))
plt.imshow(C, cmap="viridis", vmin=-1, vmax=1)
plt.colorbar(label="Correlation")
plt.title("3×3 patch correlation matrix")
plt.tight_layout()
plt.show()

# ===============================
# 7. Final correlation image
# ===============================
plt.figure(figsize=(6, 6))
plt.imshow(correlation_map, cmap="inferno")
plt.colorbar(label="Mean pairwise correlation")
plt.title("Local 3×3 pixel correlation map")
plt.axis("off")
plt.tight_layout()
plt.show()


In [18]:
summary_images = np.stack([img, img, coherence_image], axis=0).astype(np.float32)

In [ ]:


print("Computing local correlations movie with RAM FILE...")
cn = local_correlations_movie_offline(mc.mmap_file[0], fr=fr, window=fr*4, stride=fr*4, winSize_baseline=fr*2, remove_baseline=True,
                                    gaussian_blur = gaussian_blur, dview=dview).max(axis=0)
#55 s with local_correlations_movie_offline
print("Done.")
##

img_corr = (cn - np.mean(cn))/np.std(cn)
summary_images = np.stack([img, img, img_corr], axis=0).astype(np.float32)


Computing local correlations movie with RAM FILE...
Done.
C:\Users\ICNLab\caiman_data\testdata\testdata\FOV3_T3new\FOV3_T3_corr.tif
(512, 512, 3)
Saved: C:\Users\ICNLab\caiman_data\testdata\testdata\FOV3_T3new\FOV3_T3_py.png
Running Mask R-CNN inference...

Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        200
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              crop
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM          

     1644364 [deprecation.py:            new_func():554][24200] From c:\Users\ICNLab\anaconda3\envs\caiman\lib\site-packages\tensorflow\python\util\deprecation.py:629: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Use fn_output_signature instead


Processing 1 images
image                    shape: (512, 512, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -91.11000  max:  168.24000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  512.00000  int32
anchors                  shape: (1, 65472, 4)         min:   -0.04428  max:    1.01297  float32


c:\Users\ICNLab\anaconda3\envs\caiman\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


MADE FIGURE


In [19]:

cm.movie(summary_images).save(fname[:-5]+'_summary_images.tif')

plt.figure()
plt.imshow(summary_images[0], cmap='gray')
plt.axis('off')
plt.savefig(fname[:-4]+'_mean.tif', format='tif', bbox_inches='tight', pad_inches=0)
plt.close()

plt.figure()
plt.imshow(summary_images[2], cmap='gray')
plt.axis('off')
plt.savefig(fname[:-4]+'_corr.tif', format='tif', bbox_inches='tight', pad_inches=0)
plt.close()

img = summary_images.transpose([1, 2, 0])


print(fname[:-4]+'_corr.tif')
height, width = img.shape[:2]
print(img.shape)

# --------------------------------------------------------------
# Extract channels like MATLAB
# --------------------------------------------------------------
R = img[:, :, 0]
B = img[:, :, 2]

# --------------------------------------------------------------
# MATLAB-style normalization (mat2gray + uint8)
# --------------------------------------------------------------
def normalize_like_matlab(x):
    x = x.astype(np.float64)
    mn = x.min()
    mx = x.max()
    x = (x - mn) / (mx - mn + 1e-12)

    # MATLAB uint8 applies rounding, not floor
    x = np.round(255 * x).astype(np.uint8)
    return x

R_norm = normalize_like_matlab(R)
B_norm = normalize_like_matlab(B)

# --------------------------------------------------------------
# Build MATLAB-equivalent RGB (R,R,B)
# --------------------------------------------------------------
rgb = np.stack([R_norm, R_norm, B_norm], axis=2).astype(np.uint8)

# --------------------------------------------------------------
# Save as PNG (MATLAB-compatible pixel data)
# --------------------------------------------------------------
outname = fname[:-4] + "_py.png"
Image.fromarray(rgb).save(outname)

print("Saved:", outname)
img = rgb.copy()



##
print("Running Mask R-CNN inference...")
weights_path="C:/Users/ICNLab/caiman_data/testdata/testdata/mask_rcnn_neuron_0012.h5"
#download_model('mask_rcnn')
#ROIs, r = utils.mrcnn_inference(img, size_range=[0, 40], weights_path=weights_path, display_result=True)
r = utils.mrcnn_inference(img, size_range=[0, 40], weights_path=weights_path, display_result=True)
ROIs = r['masks'].transpose([2, 0, 1])
cm.movie(ROIs).save(fname[:-4]+'newmrcnn_ROIs.hdf5')

fig, axs = plt.subplots(1, 2)
axs[0].imshow(summary_images[1])
axs[1].imshow(ROIs.sum(0))
axs[0].set_title('mean image')
axs[1].set_title('masks')
plt.savefig(fname[:-6] + 'newmrcnn_ROIs.png', format='png', bbox_inches='tight', pad_inches=0)
plt.close()



C:\Users\ICNLab\caiman_data\testdata\testdata\FOV1_T2RAM2\FOV1_T2_corr.tif
(512, 512, 3)
Saved: C:\Users\ICNLab\caiman_data\testdata\testdata\FOV1_T2RAM2\FOV1_T2_py.png
Running Mask R-CNN inference...

Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        200
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              crop
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHT

     4751613 [deprecation.py:            new_func():554][31880] From c:\Users\ICNLab\anaconda3\envs\caiman\lib\site-packages\tensorflow\python\util\deprecation.py:629: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Use fn_output_signature instead


Processing 1 images
image                    shape: (512, 512, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -91.11000  max:  168.24000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  512.00000  int32
anchors                  shape: (1, 65472, 4)         min:   -0.04428  max:    1.01297  float32


c:\Users\ICNLab\anaconda3\envs\caiman\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


MADE FIGURE


In [24]:

cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
        backend='local', n_processes=None, single_thread=False, maxtasksperchild=1)

##
# ROIs = ROIs                                   # region of interests
# index = list(range(len(ROIs)))                # index of neurons
# weights = None                                # if None, use ROIs for initialization; to reuse weights check reuse weights block

# template_size = 0.02                          # half size of the window length for spike templates, default is 20 ms
# context_size = 35                             # number of pixels surrounding the ROI to censor from the background PCA
# visualize_ROI = False                         # whether to visualize the region of interest inside the context region
# hp_freq_pb = 1 / 3                            # parameter for high-pass filter to remove photobleaching
# clip = 100                                    # maximum number of spikes to form spike template
# threshold_method = 'adaptive_threshold'       # adaptive_threshold or simple
# min_spikes= 10                                # minimal spikes to be found
# pnorm = 0.5                                   # a variable deciding the amount of spikes chosen for adaptive threshold method
# threshold = 2                                 # threshold for finding spikes only used in simple threshold method, Increase the threshold to find less spikes
# do_plot = False                               # plot detail of spikes, template for the last iteration
# ridge_bg= 0.05                                # ridge regression regularizer strength for background removement, larger value specifies stronger regularization
# sub_freq = 20                                 # frequency for subthreshold extraction
# weight_update = 'ridge'                       # ridge or NMF for weight update
# n_iter = 2                                    # number of iterations alternating between estimating spike times and spatial filters

#Modified parameters
template_size = 0.02                          # half size of the window length for spike templates, default is 20 ms
context_size = 35                             # number of pixels surrounding the ROI to censor from the background PCA
visualize_ROI = False                         # whether to visualize the region of interest inside the context region
hp_freq_pb = 70                          # parameter for high-pass filter to remove photobleaching
clip = 100                                    # maximum number of spikes to form spike template
threshold_method = 'adaptive_threshold'       # adaptive_threshold or simple
min_spikes= 10                                # minimal spikes to be found
pnorm = 0.1                             # a variable deciding the amount of spikes chosen for adaptive threshold method
threshold = 2                                 # threshold for finding spikes only used in simple threshold method, Increase the threshold to find less spikes
do_plot = False                               # plot detail of spikes, template for the last iteration
ridge_bg= 0.05                                # ridge regression regularizer strength for background removement, larger value specifies stronger regularization
sub_freq = 20                                 # frequency for subthreshold extraction
weight_update = 'ridge'                       # ridge or NMF for weight update
n_iter = 2                                    # number of iterations alternating between estimating spike times and spatial filters




opts_dict={'fnames': ram_path,   #'fnames': fname_new,
        'ROIs': ROIs,
        'index': index,
        'weights': weights,
        'template_size': template_size,
        'context_size': context_size,
        'visualize_ROI': visualize_ROI,
        'hp_freq_pb': hp_freq_pb,
        'clip': clip,
        'threshold_method': threshold_method,
        'min_spikes':min_spikes,
        'pnorm': pnorm,
        'threshold': threshold,
        'do_plot':do_plot,
        'ridge_bg':ridge_bg,
        'sub_freq': sub_freq,
        'weight_update': weight_update,
        'n_iter': n_iter}

opts.change_params(params_dict=opts_dict);

vpy = VOLPY(n_processes=n_processes, dview=dview, params=opts)

print("Running VOLPY fit...")
vpy.fit(n_processes=n_processes, dview=dview)
#takes a while to run
print("Done.")

# Visualize spatial footprints and traces
print(np.where(vpy.estimates['locality'])[0])    # neurons that pass locality test
# idx = np.where(vpy.estimates['locality'] > 0)[0]
# utils.view_components(vpy.estimates, img_corr, idx)


##

# Reconstructed movie
# flip_signal = True    
# mv_all = utils.reconstructed_movie(vpy.estimates.copy(), fnames=mc.mmap_file,
#                                         idx=idx, scope=(0,1000), flip_signal=flip_signal)
#mv_all.play(fr=40, magnification=3)

##
vpy.estimates['ROIs'] = ROIs
save_name = fname[:-4]+'new_volpy'
np.save(save_name, vpy.estimates)

cm.stop_server(dview=dview)
log_files = glob.glob('*_LOG_*')
for log_file in log_files:
    os.remove(log_file)

print("Saved VOLPY estimates to:", save_name + '.npy')


Running VOLPY fit...
Starting VOLPY spike detection...
Done.
[  2   3   4   5   6   8   9  10  11  13  14  16  18  19  20  21  24  25
  26  28  31  32  35  36  38  39  47  48  50  51  52  53  54  55  57  64
  68  70  74  75  76  77  83  84  88  89  91  92  93  98 100 104 105 106]
Saved VOLPY estimates to: C:\Users\ICNLab\caiman_data\testdata\testdata\FOV1_T2RAM2\FOV1_T2new_volpy.npy


In [27]:
print(vpy.estimates.keys())
print(len(vpy.estimates['spikes']))
print(vpy.estimates['snr']) 

#print length of each key's data:
for key in vpy.estimates.keys():
    print(f"{key}: {len(vpy.estimates[key])}")

#print number of neurons with snr > 3
snr_threshold = 3.0
high_snr_neurons = np.sum(vpy.estimates['snr'] > snr_threshold)
print(f"Number of neurons with SNR > {snr_threshold}: {high_snr_neurons}")



AttributeError: 'dict' object has no attribute 'estimates'

In [26]:

##
vpy = vpy.estimates
try:
    # %% plotting all traces

    num_frames = np.max(vpy['dFF'].shape)
    dur = num_frames/640
    vpy['cellID'] = []
    vpy['raster'] = np.zeros_like(vpy['dFF'])
    vpy['firing_rate'] = np.zeros_like(vpy['dFF'])

    for i in range(vpy['dFF'].shape[0]-1):
        vpy['raster'][i,vpy['spikes'][i]] = 1
        vpy['firing_rate'][i] = savgol_filter(np.convolve(vpy['raster'][i]*640,np.ones(32)/32,mode='same'),64,1)

        if np.sqrt(np.var(vpy['templates'][i], ddof=1))>0.5:
            vpy['cellID'].append(i)

    if len(vpy['cellID'])>0:
        dFF = np.array(vpy['dFF']).astype(float)
        R = np.corrcoef(dFF)
        r = np.array(np.where(np.triu(R,1)>0.7))
        for i in range(0,r.shape[1]):
            if np.max(dFF[r[0][i]]) < np.max(dFF[r[1][i]]):
                r[1][i] = r[0][i]

        vpy['cellID'] = [x for x in vpy['cellID'] if x not in r[1]]

    cells = np.array(vpy['cellID'])
    time = np.arange(0,dur,1/640)

    fig = plt.figure(figsize=(8.0, 11.0), facecolor='w',constrained_layout=True)
    spec = fig.add_gridspec(ncols=3, nrows=5, width_ratios=[1,1,1], height_ratios=[2, 5,1,1,1])
    ax1 = fig.add_subplot(spec[0, 0])
    ax2 = fig.add_subplot(spec[0, 1])
    ax_text = fig.add_subplot(spec[0, 2],facecolor='w')
    ax3 = fig.add_subplot(spec[1, :],facecolor='w')
    ax4 = fig.add_subplot(spec[4, :],facecolor='w')
    ax5 = fig.add_subplot(spec[2, :],facecolor='w')
    ax5r = ax5.twinx()
    ax6 = fig.add_subplot(spec[3, :],facecolor='w')
    #ax7 = fig.add_subplot(spec[4, :],facecolor='w')

    ax1.imshow(img[:,:,1], cmap='gray')
    ax2.imshow(img[:,:,2], cmap='gray')
    ax1.set_title('Mean image',color='k',fontsize=14)
    ax2.set_title('Corr image',color='k',fontsize=14)
    ax1.set_axis_off()
    ax2.set_axis_off()
    ax_text.set_axis_off()

    llim = 0
    if len(cells)>0:
        pos_cells = []
        neg_cells = []
        b, a = butter(1, [1.5, 100], fs=640, btype='band')
        k = 1
        for i in range(0, len(cells)):
            if ''.join(vpy['polarity'][cells[i]]) in 'negative':
                color = '#9AAB3A'
                mult = -1
                neg_cells.append(cells[i])
            else:
                color = '#54A0A8'
                mult = 1
                pos_cells.append(cells[i])
            y = np.array(lfilter(b,a,stats.zscore(np.array(vpy['dFF'][cells[i]] * mult * 100,dtype=np.float32))) + ((k - 1) * 8)).reshape(1,num_frames)
            ax3.plot(llim+time,y[0,:],color, linewidth=0.3)
            ax3.plot(llim+time[vpy['spikes'][cells[i]]],np.max(y)*np.ones(vpy['spikes'][cells[i]].shape[0]),"|",color='firebrick',markersize=2)
            k = k + 1


        if len(pos_cells)>0:
            mean_fr_pos = np.mean(vpy['firing_rate'][pos_cells,:], axis=0)
            sem_pos = stats.sem(np.array(vpy['firing_rate'][pos_cells,:],dtype=np.float32), axis=0)
            ax5r.plot(llim+time, np.array(mean_fr_pos,dtype='float32').ravel(), label='Mean firing rate', color='#54A0A8',linewidth=0.3)
            ax5r.fill_between(llim+time, np.array(mean_fr_pos - sem_pos,dtype='float32').ravel(), np.array(mean_fr_pos + sem_pos,dtype='float32'), color='#54A0A8', alpha=0.3, label='SEM')
            ax5.set_ylabel('Firing rate (Hz)',color='#54A0A8',fontsize=12)
            ax5r.tick_params(axis ='y', labelcolor = '#54A0A8')
        if len(neg_cells)>0:
            mean_fr_neg = np.mean(vpy['firing_rate'][neg_cells,:], axis=0)
            sem_neg = stats.sem(np.array(vpy['firing_rate'][neg_cells,:],dtype=np.float32), axis=0)
            ax5.plot(llim+time, np.array(mean_fr_neg,dtype='float32').ravel(), label='Mean firing rate', color='#9AAB3A',linewidth=0.3)
            ax5.fill_between(llim+time, np.array(mean_fr_neg - sem_neg,dtype='float32').ravel(), np.array(mean_fr_neg + sem_neg,dtype='float32'), color='#9AAB3A', alpha=0.3, label='SEM')
            ax5.set_ylabel('Firing rate (Hz)',color='#9AAB3A',fontsize=12)
            ax5r.tick_params(axis ='y', labelcolor = '#9AAB3A')

    wheel_mat = os.path.dirname(fname) + '\\Wheel.mat'
    if os.path.exists(wheel_mat):
        wheel=mat73.loadmat(wheel_mat)
        if 'behavior' in wheel:
            ax4.plot(wheel['behavior'][:,0],wheel['behavior'][:,1],'r',linewidth=1.2)
            if wheel['behavior'].shape[1]>2:
                ax4.plot(wheel['behavior'][:,0],wheel['behavior'][:,2],'k',linewidth=1)
            ax4.set_ylabel('Behavior',color='k',fontsize=12)
            ax4.set_yticks([-1,0,1])
            ax4.set_ylim([-2,2])

        if wheel['data_time'].any():
            whl_time = np.arange(0,np.max(wheel['data_time']),1/640)
            wheel_interp = np.interp(whl_time, wheel['data_time'], wheel['data_pos'])
            speed = np.zeros_like(wheel_interp)
            for i in range(0,len(whl_time)-1):
                speed[i] = (wheel_interp[i+1]-wheel_interp[i])/(whl_time[i+1]-whl_time[i])

            #speed[speed>100] = 0
            #speed[speed<0] = 0
            speed = savgol_filter(speed,64,1)
            ax6.plot(whl_time,speed,'k',linewidth=1)
            ax6.set_ylabel('Speed (cm/s)',color='k',fontsize=12)
            #ax7.plot(whl_time,speed,'w',linewidth=1)
            #ax7.set_ylabel('Speed (cm/s)',color='k',fontsize=12)

        ax_text.text(0.5, 0.8, 'Mouse = ' + wheel['mouse'], color='k',fontsize=10, ha='center')
        ax_text.text(0.5, 0.4, 'Stimulus = ' + wheel['stimulus'], color='k',fontsize=10, ha='center')
        ax_text.text(0.5, 0.6, 'Date = ' + str(np.array(wheel['currentdate'],dtype='int32')), color='k',fontsize=10, ha='center')
        #ax_text.text(0.5, 0.2, 'File = ' + wheel['file'], color='w',fontsize=10, ha='center')
        if wheel['stimulus']=='Map' and 'rand_num' in wheel:
            ax_text.text(0.5, 0, 'Field = ' + " ".join(str(x) for x in wheel['rand_num'].astype(int)), color='k',fontsize=10, ha='center')
        elif wheel['stimulus']=='Tuning' and 'rand_num' in wheel:
            ax_text.text(0.5, 0, 'Orientation = ' + " ".join(str(x) for x in wheel['rand_num'].astype(int)), color='k',fontsize=10, ha='center')
        elif wheel['stimulus']=='Tuning' and 'rand_num' in wheel:
            ax_text.text(0.5, 0, 'Orientation = ' + " ".join(str(x) for x in wheel['rand_num'].astype(int)), color='k',fontsize=10, ha='center')
    else:
        print("Wheel data does not exist")


    for ax in [ax3,ax4,ax5,ax6]:
        ax.tick_params(color='black', labelcolor='black')
        ax.set_xlabel('Time (sec)',color='k',fontsize=12)
        ax.set_xlim([llim,llim+dur])
        for spine in ax.spines.values():
            spine.set_edgecolor('black')
    ax3.set_title('dFF',color='k',fontsize=14)
    ax3.set_ylabel(r'$\Delta$F/F (%)',color='k',fontsize=12)


    fig.savefig(fname[:-4] + '_volpy.pdf')
    #plt.close('all')
    
    print("Saved VOLPY figure to:", fname[:-4] + '_volpy.pdf')

    print("Saving VOLPY data to MAT file...")
    vpy['ROIs'] = ROIs
    #vpy['rect'] = r['rois']
    vpy['img'] = img
    del vpy['rawROI']
    #scipy.io.savemat(fname[:-4] + '_volpy.mat', {'vpy': vpy}, format='5', do_compression=True)
    

    print("Converting data types for fast saving...")

    # Keys identified from inspection output that need fixing
    keys_to_convert_float = [
        't', 'ts', 't_rec', 't_sub', 'templates', 'snr', 
        'thresh', 'weights', 'locality', 'context_coord', 'F0', 'dFF', 
        'raster', 'firing_rate'
    ]

    keys_to_convert_int = [
        'num_spikes'
    ]

    # Process float conversions
    for key in keys_to_convert_float:
        if key in vpy and vpy[key].dtype == object:
            try:
                # Attempt a direct conversion to float32 (fastest for scientific data)
                vpy[key] = np.array(vpy[key], dtype=np.float32)
                print(f"  Converted '{key}' to float32 array.")
            except ValueError:
                print(f"  Could not convert '{key}' to standard array dtype. Keeping as object array.")

    # Process integer conversions
    for key in keys_to_convert_int:
        if key in vpy and vpy[key].dtype == object:
            try:
                vpy[key] = np.array(vpy[key], dtype=np.int32)
                print(f"  Converted '{key}' to int32 array.")
            except ValueError:
                print(f"  Could not convert '{key}' to int32 array. Keeping as object array.")

    # Handle variables that are inherently irregular lists that MUST be object arrays in Python, 
    # but we ensure they are clean for saving.

    # Handle 'mean_im', 'cell_n', 'polarity' (irregular shapes/strings)
    for key in ['mean_im', 'cell_n', 'polarity']:
        if key in vpy and vpy[key].dtype == object:
            vpy[key] = np.array(vpy[key], dtype=object) # Ensure they are formally object arrays

    # Handle spikes and low_spikes. The try/except handles the 'bool is not iterable' error.
    if vpy['spikes'].dtype == object:
        vpy['spikes'] = np.array([list(x) for x in vpy['spikes']], dtype=object)
        
    if vpy['low_spikes'].dtype == object:
        try:
            # This was causing the TypeError because it was actually a boolean array
            vpy['low_spikes'] = np.array([list(x) for x in vpy['low_spikes']], dtype=object)
        except TypeError:
            # If it's a bool array, just make sure it's saved as a clean boolean array
            vpy['low_spikes'] = np.array(vpy['low_spikes'], dtype=bool) 


    print("Data type conversion complete.")
    
    scipy.io.savemat(fname[:-4] + '_volpy.mat', {'vpy': vpy}, format='5', do_compression=True)
    print("Saved VOLPY data to:", fname[:-4] + '_2volpy.mat')

    # vpy.estimates['params'] = opts
    # save_name = f'volpy_{os.path.split(fnames)[1][:-5]}_{threshold_method}'
    # np.save(fnames[:-4] + '_volpy.npy', vpy.estimates)

    #del vpy
    # %% STOP CLUSTER and clean up log files


    log_files = glob.glob('*_LOG_*')
    for log_file in log_files:
        os.remove(log_file)
except ValueError:
    traceback.print_exc()
    print("No volpy data was saved")

Saved VOLPY figure to: C:\Users\ICNLab\caiman_data\testdata\testdata\FOV1_T2RAM2\FOV1_T2_volpy.pdf
Saving VOLPY data to MAT file...
Converting data types for fast saving...
  Converted 't' to float32 array.
  Converted 'ts' to float32 array.
  Converted 't_rec' to float32 array.
  Converted 't_sub' to float32 array.
  Converted 'templates' to float32 array.
  Converted 'snr' to float32 array.
  Converted 'thresh' to float32 array.
  Converted 'weights' to float32 array.
  Converted 'locality' to float32 array.
  Converted 'context_coord' to float32 array.
  Converted 'F0' to float32 array.
  Converted 'dFF' to float32 array.
  Converted 'raster' to float32 array.
  Converted 'firing_rate' to float32 array.
  Converted 'num_spikes' to int32 array.
Data type conversion complete.
Saved VOLPY data to: C:\Users\ICNLab\caiman_data\testdata\testdata\FOV1_T2RAM2\FOV1_T2_2volpy.mat


In [51]:
print(vpy.keys())
print(len(vpy['spikes']))
print(vpy['snr']) 

#print length of each key's data:
for key in vpy.keys():
    print(f"{key}: {len(vpy[key])}")
#print the coordinates of each neuron
for i in range(len(vpy['spikes'])):
    print(f"Neuron {i}: locality = {vpy['locality'][i]}, context_coord = {vpy['context_coord'][i]}")    

#print number of neurons with snr > 3
snr_threshold = 5.0
high_snr_neurons = np.sum(vpy['snr'] > snr_threshold)
print(f"Number of neurons with SNR > {snr_threshold}: {high_snr_neurons}")

#display img (512x512x3) stored in dictionary vpy
# ===============================
plt.figure(figsize=(6, 6))
plt.imshow(vpy['img'], cmap='gray')
plt.title('VOLPY mean image')
plt.axis("off")
plt.tight_layout()
plt.show()

#overlay each context coordinate box on the img (cood is [(x1,y1), (x2,y2)] format)
plt.figure(figsize=(6, 6))      
plt.imshow(vpy['img'], cmap='gray')
for coord in vpy['context_coord']:
    (x1, y1), (x2, y2) = coord
    rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, edgecolor='red', facecolor='none', linewidth=1)
    plt.gca().add_patch(rect)
plt.title('VOLPY context regions')
plt.axis("off")
plt.tight_layout()
plt.show()

dict_keys(['mean_im', 'cell_n', 't', 'ts', 't_rec', 't_sub', 'spikes', 'low_spikes', 'num_spikes', 'templates', 'snr', 'thresh', 'weights', 'locality', 'context_coord', 'F0', 'dFF', 'polarity', 'ROIs', 'cellID', 'raster', 'firing_rate', 'img'])
108
[ 2.2832398 -3.5445855 11.29796    7.738044   3.205537   9.663431
  1.1494905 -2.0245862  3.4733834  3.959727   2.9571002 16.407326
 -1.3928297 13.05479    7.9565024 -4.344558   3.3550951 -3.8677661
 11.033477   3.3255389 11.836677   1.7227821 -2.2650077 -3.58393
  1.7308702  2.0075724  8.213702   4.116227   2.2615964 -1.8638135
 -3.7907813  2.8585284 15.351904  -0.7349891 -2.5207229  1.5922941
  4.4084163 -2.1523283  7.1230984  2.0641942 -3.1359792 -1.7648456
 -3.37024    1.9081684  3.0166798 -3.6531832  6.004449   5.4073153
  4.149292   6.848043   3.460325   9.769232   3.7711208  1.6533691
  2.4246879  1.8729694  6.7031975  9.742022  -3.7846487 -2.3734438
 -3.5294197 12.358633   1.8334635 -4.420713   7.8524823  2.544922
 -7.6872716 -3.9934

In [ ]:
idx = np.where(vpy['locality'] > 0)[0]
flip_signal = True

mv_all = utils.reconstructed_movie(
    vpy.copy(),
    fnames=mc.mmap_file,
    idx=idx,
    scope=(0, 1000),
    flip_signal=flip_signal
)


100%|██████████| 1/1 [00:08<00:00,  8.16s/it]


TypeError: timeseries.save() got an unexpected keyword argument 'fr'

In [38]:
out_dir = os.path.dirname(fname)
out_path = os.path.join(out_dir, 'reconstructed_movie.mp4')

writer = imageio.get_writer(
    out_path,
    fps=40,
    codec='libx264',
    pixelformat='yuv420p'
)

for frame in mv_all:
    f = frame.astype(np.float32)
    f -= f.min()
    f /= f.max() + 1e-8
    f = (255 * f).astype(np.uint8)
    writer.append_data(f)

writer.close()

In [ ]:
import os
import imageio.v2 as imageio
import numpy as np

# --- parameters ---
orig_fps = 40          # original frame rate
seconds = 5
slowdown = 2.0         # 2x slower (increase for more slow-mo)

gif_fps = orig_fps / slowdown
n_frames = int(seconds * orig_fps)

# --- output path ---
out_dir = os.path.dirname(fname)
gif_path = os.path.join(out_dir, 'reconstructed_last5s_slow.gif')

# --- select last frames ---
frames = mv_all[-n_frames:]

with imageio.get_writer(
    gif_path,
    mode='I',
    fps=gif_fps,
    loop=0        # 0 = loop forever
) as writer:

    for frame in frames:
        f = frame.astype(np.float32)

        # crop to right third for visibility
        h, w = f.shape
        f = f[:, 2 * w // 3 : w]

        # normalize for visibility
        f -= f.min()
        f /= f.max() + 1e-8
        f = (255 * f).astype(np.uint8)

        writer.append_data(f)

print(f"Saved GIF to: {gif_path}")


Saved GIF to: C:\Users\ICNLab\caiman_data\testdata\testdata\FOV1_T2RAM2\reconstructed_last5s_slow.gif


Component:1


Traceback (most recent call last):
  File "c:\Users\ICNLab\anaconda3\envs\caiman\lib\site-packages\matplotlib\cbook.py", line 298, in process
    func(*args, **kwargs)
  File "c:\Users\ICNLab\anaconda3\envs\caiman\lib\site-packages\matplotlib\widgets.py", line 592, in <lambda>
    return self._observers.connect('changed', lambda val: func(val))
  File "C:\Users\ICNLab\CaImAn_GV\caiman\source_extraction\volpy\utils.py", line 364, in update
    1.05 * np.max(estimates['t'][idx][i]) * np.ones(estimates['spikes'][idx[i]].shape),
AttributeError: 'list' object has no attribute 'shape'


Component:2


Traceback (most recent call last):
  File "c:\Users\ICNLab\anaconda3\envs\caiman\lib\site-packages\matplotlib\cbook.py", line 298, in process
    func(*args, **kwargs)
  File "c:\Users\ICNLab\anaconda3\envs\caiman\lib\site-packages\matplotlib\widgets.py", line 592, in <lambda>
    return self._observers.connect('changed', lambda val: func(val))
  File "C:\Users\ICNLab\CaImAn_GV\caiman\source_extraction\volpy\utils.py", line 364, in update
    1.05 * np.max(estimates['t'][idx][i]) * np.ones(estimates['spikes'][idx[i]].shape),
AttributeError: 'list' object has no attribute 'shape'


...

Component:52


Traceback (most recent call last):
  File "c:\Users\ICNLab\anaconda3\envs\caiman\lib\site-packages\matplotlib\cbook.py", line 298, in process
    func(*args, **kwargs)
  File "c:\Users\ICNLab\anaconda3\envs\caiman\lib\site-packages\matplotlib\widgets.py", line 592, in <lambda>
    return self._observers.connect('changed', lambda val: func(val))
  File "C:\Users\ICNLab\CaImAn_GV\caiman\source_extraction\volpy\utils.py", line 364, in update
    1.05 * np.max(estimates['t'][idx][i]) * np.ones(estimates['spikes'][idx[i]].shape),
AttributeError: 'list' object has no attribute 'shape'


Component:53


Traceback (most recent call last):
  File "c:\Users\ICNLab\anaconda3\envs\caiman\lib\site-packages\matplotlib\cbook.py", line 298, in process
    func(*args, **kwargs)
  File "c:\Users\ICNLab\anaconda3\envs\caiman\lib\site-packages\matplotlib\widgets.py", line 592, in <lambda>
    return self._observers.connect('changed', lambda val: func(val))
  File "C:\Users\ICNLab\CaImAn_GV\caiman\source_extraction\volpy\utils.py", line 364, in update
    1.05 * np.max(estimates['t'][idx][i]) * np.ones(estimates['spikes'][idx[i]].shape),
AttributeError: 'list' object has no attribute 'shape'


Component:54
Component:54
Component:54


In [37]:
print(out_path)

C:\Users\ICNLab\caiman_data\testdata\testdata\FOV1_T2RAM2\reconstructed_movie.avi


In [ ]:
    plt.close('all')
    
    print("Saved VOLPY figure to:", fname[:-4] + '_volpy.pdf')

    print("Saving VOLPY data to MAT file...")
    vpy['ROIs'] = ROIs
    #vpy['rect'] = r['rois']
    vpy['img'] = img
    del vpy['rawROI']
    #scipy.io.savemat(fname[:-4] + '_volpy.mat', {'vpy': vpy}, format='5', do_compression=True)
    

    print("Converting data types for fast saving...")

    # Keys identified from inspection output that need fixing
    keys_to_convert_float = [
        't', 'ts', 't_rec', 't_sub', 'templates', 'snr', 
        'thresh', 'weights', 'locality', 'context_coord', 'F0', 'dFF', 
        'raster', 'firing_rate'
    ]

    keys_to_convert_int = [
        'num_spikes'
    ]

    # Process float conversions
    for key in keys_to_convert_float:
        if key in vpy and vpy[key].dtype == object:
            try:
                # Attempt a direct conversion to float32 (fastest for scientific data)
                vpy[key] = np.array(vpy[key], dtype=np.float32)
                print(f"  Converted '{key}' to float32 array.")
            except ValueError:
                print(f"  Could not convert '{key}' to standard array dtype. Keeping as object array.")

    # Process integer conversions
    for key in keys_to_convert_int:
        if key in vpy and vpy[key].dtype == object:
            try:
                vpy[key] = np.array(vpy[key], dtype=np.int32)
                print(f"  Converted '{key}' to int32 array.")
            except ValueError:
                print(f"  Could not convert '{key}' to int32 array. Keeping as object array.")

    # Handle variables that are inherently irregular lists that MUST be object arrays in Python, 
    # but we ensure they are clean for saving.

    # Handle 'mean_im', 'cell_n', 'polarity' (irregular shapes/strings)
    for key in ['mean_im', 'cell_n', 'polarity']:
        if key in vpy and vpy[key].dtype == object:
            vpy[key] = np.array(vpy[key], dtype=object) # Ensure they are formally object arrays

    # Handle spikes and low_spikes. The try/except handles the 'bool is not iterable' error.
    if vpy['spikes'].dtype == object:
        vpy['spikes'] = np.array([list(x) for x in vpy['spikes']], dtype=object)
        
    if vpy['low_spikes'].dtype == object:
        try:
            # This was causing the TypeError because it was actually a boolean array
            vpy['low_spikes'] = np.array([list(x) for x in vpy['low_spikes']], dtype=object)
        except TypeError:
            # If it's a bool array, just make sure it's saved as a clean boolean array
            vpy['low_spikes'] = np.array(vpy['low_spikes'], dtype=bool) 


    print("Data type conversion complete.")
    
    scipy.io.savemat(fname[:-4] + '_volpy.mat', {'vpy': vpy}, format='5', do_compression=True)
    print("Saved VOLPY data to:", fname[:-4] + '_volpy.mat')

    # vpy.estimates['params'] = opts
    # save_name = f'volpy_{os.path.split(fnames)[1][:-5]}_{threshold_method}'
    # np.save(fnames[:-4] + '_volpy.npy', vpy.estimates)

    del vpy
    # %% STOP CLUSTER and clean up log files


    log_files = glob.glob('*_LOG_*')
    for log_file in log_files:
        os.remove(log_file)
except ValueError:
    traceback.print_exc()
    print("No volpy data was saved")

In [16]:
import numpy as np

def inspect_data_structure(data_dict, name='Data Structure', indent=''):
    """
    Recursively inspects a dictionary intended for scipy.io.savemat 
    and prints the types and shapes of contained variables.
    """
    print(f"{indent}--- Inspecting: {name} ---")
    
    # Check top-level type
    if not isinstance(data_dict, dict):
        print(f"{indent}WARNING: Top-level object is not a dictionary. Type: {type(data_dict)}")
        return

    for key, value in data_dict.items():
        # Handle NumPy arrays
        if isinstance(value, np.ndarray):
            shape_str = f"Shape: {value.shape}"
            dtype_str = f"dtype: {value.dtype}"
            print(f"{indent}|- '{key}': NumPy Array | {shape_str} | {dtype_str}")
        
        # Handle basic Python lists/tuples
        elif isinstance(value, (list, tuple)):
            if len(value) > 0 and isinstance(value[0], (np.ndarray, dict, list, tuple)):
                # If it's a list of complex objects, print summary of first element and recurse
                print(f"{indent}|- '{key}': List/Tuple ({len(value)} items). Inspecting first item:")
                # Create a temporary dict just to use the recursive function logic
                inspect_data_structure({'Item_0': value[0]}, name=f"List Item 0 (inside '{key}')", indent=indent + '  ')
            else:
                # If it's a simple list (e.g., strings, numbers)
                val_type = type(value[0]) if len(value) > 0 else 'Empty'
                print(f"{indent}|- '{key}': List/Tuple ({len(value)} items). Contains: {val_type}")

        # Handle nested dictionaries (MATLAB structures)
        elif isinstance(value, dict):
            print(f"{indent}|- '{key}': Nested Dictionary. Recursing:")
            inspect_data_structure(value, name=f"Nested Dict '{key}'", indent=indent + '  ')
            
        # Handle other types
        else:
            print(f"{indent}|- '{key}': Other Type: {type(value)}")

    print(f"{indent}--- End of Inspection: {name} ---")



In [18]:
inspect_data_structure({'vpy': vpy}) 


--- Inspecting: Data Structure ---
|- 'vpy': Nested Dictionary. Recursing:
  --- Inspecting: Nested Dict 'vpy' ---
  |- 'mean_im': NumPy Array | Shape: (76,) | dtype: object
  |- 'cell_n': NumPy Array | Shape: (76,) | dtype: object
  |- 't': NumPy Array | Shape: (76, 12800) | dtype: object
  |- 'ts': NumPy Array | Shape: (76, 12800) | dtype: object
  |- 't_rec': NumPy Array | Shape: (76, 12800) | dtype: object
  |- 't_sub': NumPy Array | Shape: (76, 12800) | dtype: object
  |- 'spikes': NumPy Array | Shape: (76,) | dtype: object
  |- 'low_spikes': NumPy Array | Shape: (76,) | dtype: object
  |- 'num_spikes': NumPy Array | Shape: (76, 3) | dtype: object
  |- 'templates': NumPy Array | Shape: (76, 25) | dtype: object
  |- 'snr': NumPy Array | Shape: (76,) | dtype: object
  |- 'thresh': NumPy Array | Shape: (76,) | dtype: object
  |- 'weights': NumPy Array | Shape: (76, 512, 512) | dtype: object
  |- 'locality': NumPy Array | Shape: (76,) | dtype: object
  |- 'context_coord': NumPy Array 

In [20]:
import numpy as np

print("Converting data types for fast saving...")

# Keys identified from inspection output that need fixing
keys_to_convert_float = [
    't', 'ts', 't_rec', 't_sub', 'templates', 'snr', 
    'thresh', 'weights', 'locality', 'context_coord', 'F0', 'dFF', 
    'raster', 'firing_rate'
]

keys_to_convert_int = [
    'num_spikes'
]

# Process float conversions
for key in keys_to_convert_float:
    if key in vpy and vpy[key].dtype == object:
        try:
            # Attempt a direct conversion to float32 (fastest for scientific data)
            vpy[key] = np.array(vpy[key], dtype=np.float32)
            print(f"  Converted '{key}' to float32 array.")
        except ValueError:
            print(f"  Could not convert '{key}' to standard array dtype. Keeping as object array.")

# Process integer conversions
for key in keys_to_convert_int:
    if key in vpy and vpy[key].dtype == object:
        try:
            vpy[key] = np.array(vpy[key], dtype=np.int32)
            print(f"  Converted '{key}' to int32 array.")
        except ValueError:
            print(f"  Could not convert '{key}' to int32 array. Keeping as object array.")

# Handle variables that are inherently irregular lists that MUST be object arrays in Python, 
# but we ensure they are clean for saving.

# Handle 'mean_im', 'cell_n', 'polarity' (irregular shapes/strings)
for key in ['mean_im', 'cell_n', 'polarity']:
    if key in vpy and vpy[key].dtype == object:
         vpy[key] = np.array(vpy[key], dtype=object) # Ensure they are formally object arrays

# Handle spikes and low_spikes. The try/except handles the 'bool is not iterable' error.
if vpy['spikes'].dtype == object:
    vpy['spikes'] = np.array([list(x) for x in vpy['spikes']], dtype=object)
    
if vpy['low_spikes'].dtype == object:
    try:
        # This was causing the TypeError because it was actually a boolean array
        vpy['low_spikes'] = np.array([list(x) for x in vpy['low_spikes']], dtype=object)
    except TypeError:
        # If it's a bool array, just make sure it's saved as a clean boolean array
        vpy['low_spikes'] = np.array(vpy['low_spikes'], dtype=bool) 


print("Data type conversion complete.")


Converting data types for fast saving...
Data type conversion complete.


In [21]:
# Insert this line right after the conversion script you just ran:

scipy.io.savemat(fname[:-4] + '_volpy.mat', {'vpy': vpy}, format='5', do_compression=False)

print("Saved VOLPY data to:", fname[:-4] + '_volpy.mat')

# ... the rest of your original script ...


Saved VOLPY data to: C:\Users\ICNLab\caiman_data\testdata\testdata\FOV1_T2RAM2\FOV1_T2_volpy.mat
